# Model Testing


## Setup


In [ ]:
from scripts.preprocessing.data_loader import get_train_test_splits, get_combined_dataset
from scripts.preprocessing.preprocessor import apply_minmax_scaling, select_features
from scripts.models.model_list import load_models
from scripts.models.model_testing import test_models
from IPython.display import display
from scripts.visualisations.plotting import plot_group_averaged_spatial_heatmap
from scripts.visualisations.helpers import rescale_features
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = get_train_test_splits(test_size=0.2)
X_train, y_train, train_scales = apply_minmax_scaling(X_train, y_train)
X_test, y_test, test_scales = apply_minmax_scaling(X_test, y_test)
df = get_combined_dataset(X_train, y_train, X_test, y_test)
print(f"Dataframe shape: {df.shape}")

TEST_SET, ALL_SINGLE_TARGET_MODELS, REFINED_SINGLE_TARGET_MODELS, ALL_MULTI_TARGET_MODELS, REFINED_MULTI_TARGET_MODELS = load_models()
print(f"Number of single-target models: {len(ALL_SINGLE_TARGET_MODELS)}")
print(f"Number of multi-target models: {len(ALL_MULTI_TARGET_MODELS)}")

positional_constants = ['x', 'y', 'year']
features = ['bedrock_elevation', 'precipitation', 'air_temperature', 'ocean_temperature']
targets = ['ice_thickness', 'ice_velocity', 'ice_mask']
other = ['set']
derived = [col for col in df.columns if col not in features + targets + positional_constants + other]

In [3]:
thickness_features = select_features(X_train, y_train["ice_thickness"], method='combined')
velocity_features = select_features(X_train, y_train["ice_velocity"], method='combined')
mask_features = select_features(X_train, y_train["ice_mask"], method='combined')

print(thickness_features)
print(velocity_features)
print(mask_features)

## Dataset

In [ ]:
display("Full Dataset",df)
display("Variable Measurements",df.describe())
display("Dataset Info", df.info())

print(len(positional_constants), len(features), len(derived), len(target))

display("Features", df[features].describe())
display("Target", df[target].describe())
display("Derived", df[derived].describe())

In [ ]:
df_test = df[df['set'] == 'test'].copy()
df_test = rescale_features(df_test, test_scales)

input_features = ['bedrock_elevation', 'precipitation', 'air_temperature', 'ocean_temperature']
input_titles = ['Bedrock Elevation', 'Precipitation', 'Air Temperature', 'Ocean Temperature']
input_units = ['m', 'mm/year', '°K', '°K']

derived_features = ['distance_to_pole', 'bedrock_below_sea_level', 'temperature_difference', 
                    'log_air_temperature', 'rolling_std_precipitation', 'rolling_std_air_temperature', 
                    'air_temperature_low_45', 'bedrock_slope_x', 'bedrock_slope_y', 'bedrock_slope_magnitude', 'surface_mass_balance', 'years_since_start']
derived_titles = ['Distance to Pole', 'Bedrock Below Sea Level', 'Temperature Difference', 
                    'Log Air Temperature', 'Rolling Std Precipitation', 'Rolling Std Air Temperature', 
                    'Air Temperature Low 45', 'Bedrock Slope X', 'Bedrock Slope Y', 'Bedrock Slope Magnitude', 'Surface Mass Balance', 'Years Since Start']
derived_units = ['', '', '°K', '°K', 'mm/year', '°K', '', 'm/m', 'm/m', 'm/m', 'mm/year', 'years']

target_features = ['ice_thickness', 'ice_velocity', 'ice_mask']
target_titles = ['Ice Thickness', 'Ice Velocity', 'Ice Mask']
target_units = ['m', 'm/year', '']

input_plot = plot_group_averaged_spatial_heatmap(df, input_features, input_titles, input_units, figsize=(20, 15))
input_plot.suptitle('Input Features', fontsize=20, y=1.02)
input_plot.savefig('out/images/input_features_heatmaps.png', dpi=300, bbox_inches='tight')

derived_plot = plot_group_averaged_spatial_heatmap(df, derived_features, derived_titles, derived_units, figsize=(20, 35))
derived_plot.suptitle('Derived Features', fontsize=20, y=1.02)
derived_plot.savefig('out/images/derived_features_heatmaps.png', dpi=300, bbox_inches='tight')

target_plot = plot_group_averaged_spatial_heatmap(df, target_features, target_titles, target_units, figsize=(20, 15))
target_plot.suptitle('Target Outputs', fontsize=20, y=1.02)
target_plot.savefig('out/images/target_outputs_heatmaps.png', dpi=300, bbox_inches='tight')

plt.show()

## Broad Testing


### Single-Target


In [ ]:
broad_single_target_results = test_models(ALL_SINGLE_TARGET_MODELS, multi=False, cv=5, refined=False, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

### Multi-Target


In [ ]:
broad_multi_target_results = test_models(ALL_MULTI_TARGET_MODELS, multi=True, cv=5, refined=False, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

## Refined Testing


### Single-Target


In [ ]:
refined_single_target_results = test_models(REFINED_SINGLE_TARGET_MODELS, multi=False, cv=5, refined=True, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

### Multi-Target


In [ ]:
refined_multi_target_results = test_models(REFINED_MULTI_TARGET_MODELS, multi=True, cv=5, refined=True, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

## Sequential Models
